In [367]:
import requests
import pdf2image 
import numpy as np
from PIL import Image
import pytesseract
import re
from IPython.display import display
from tqdm import tqdm
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import unidecode
import io
import PyPDF2
import bs4 as bs
import os
import string
from itertools import product

In [6]:
pattern = r" 0\\n\( (\d+ \d+ \d+ \d+) ((?:0\\n\d+ \d+ \d+ \d+ \d+ )+)0\\n\) \d+ \d+ \d+ \d+"
compiled_pattern = re.compile(pattern)

def get_num_pages(pdf):
    pdf_file = io.BytesIO(pdf.content)
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)
    return num_pages

In [382]:
pattern = re.compile(r"(\(\d+\))")
pattern2 = re.compile(r"(\d+ \d+ \d+ \d+);")

def custom_selector(tag):
    if type(tag) is bs.element.Tag:
    # Return "span" tags with a class name of "target_span"
        condition1= tag.name == "span" and "ocrx_word" in tag.get("class")
        condition2 =  tag.previousSibling == '\n' and tag.next_sibling == '\n'
        child = tag.find_all_next() 
        if child: 
            condition4 = child[0].find_all_previous()[0] == tag # checking to make sure it is first child
            condition3 = re.match(pattern, str(tag.string)) 
            return condition1 and condition2 and condition3 and condition4
    return False

fn = lambda x : 255 if x >= 127.5 else 0

def extract_formulas(image_, chapter):
    image = image_.convert('L').point(fn, mode='1')
    w,h = image.size
    hocr = pytesseract.image_to_pdf_or_hocr(image, extension='hocr')
    soup = bs.BeautifulSoup(hocr,'html.parser')
    matches = soup.find_all(custom_selector)
    im = np.asarray(image_)
    alphabet = list(string.ascii_lowercase)
    for match in matches: 
        x_0,y_0,x_1,y_1 = [int(x) for x in re.findall(pattern2, match.get("title"))[0].split(' ')]
        if x_0 < w/4: # extra check to make sure it doesn't misclassify an i for a 1
            continue
        formula_num = int(''.join(c for c in str(match.string) if c.isdigit()))
        final_image = Image.fromarray(im[y_0-120:y_1+120, :])
        fname = f'real_and_complex/{chapter}_{formula_num}.jpg'
        count = 0
        while os.path.isfile(fname): # multiple formulas named the same in the same chapter
            fname = f'real_and_complex/{chapter}_{formula_num}_' + alphabet[count] + '.jpg'
            count += 1
            if count == 26:
                alphabet = [''.join(x) for x in list(product(alphabet, alphabet))]
                count = 0
        final_image.save(fname)

In [1]:
images

NameError: name 'images' is not defined

In [240]:
books = {'principles_of_mathematical_analysis': 'https://download.tuxfamily.org/openmathdep/analysis_real/Principles_of_Mathematical_Analysis-Rudin.pdf', 
             'real_and_complex_analysis': 'https://perso.telecom-paristech.fr/decreuse/_downloads/c22155fef582344beb326c1f44f437d2/rudin.pdf', 
            'functional_analysis':'https://www.mymathscloud.com/api/download/modules/University/Textbooks/functional-analysis/Functional%20Analysis%20Rudin.pdf?id=48928539' }

In [324]:
url = books['real_and_complex_analysis']
pdf = requests.get(url, stream=True)

images = []
for first_page, last_page in tqdm([(x[0], x[-1]) for x in np.array_split(range(get_num_pages(pdf)), 10)]):
    images.append(pdf2image.convert_from_bytes(pdf.content, first_page=first_page, 
                    last_page=last_page, thread_count=10, grayscale=True, dpi = 500))
    
images = [item for row in images for item in row]

100%|███████████████████████████████████████████| 10/10 [00:50<00:00,  5.09s/it]


In [245]:
"""chapters = {1:(1,21), 2: (24,43), 3: (47,78), 4:(83,98), 5:(103, 114),\
6:(120,138), 7:(143, 165), 8:(172,196), 9:(204, 239), 10:(245, 288), 11:(300,332)}
chapters = {k:(v[0]+9, v[1]+9) for k,v in chapters.items()}
"""
chapters = {1:(5,31), 2: (33,57), 3: (61,71), 4:(76,92),\
            5:(95, 112), 6:(116,132), 7:(135, 156), 8:(160,174), 9:(178, 193), 10:(196, 227),\
            11:(231,249), 12: (253, 264), 13: (266, 276), 14: (278, 293), 15: (298, 315), \
           16: (319, 332), 17: (335, 352), 18: (356, 369), 19: (371, 383), 20: (386, 394)}
chapters = {k:(v[0]+14, v[1]+14) for k,v in chapters.items()}


In [383]:
with tqdm(total=chapters[1][1] - chapters[1][0] + 1 , position = 0, leave = True, desc = f"Processing chapter 1") as pbar: 
    for chapter, (start, end) in chapters.items():
        if chapter == 1:
            pass
        else: 
            pbar.reset(total = end - start + 1)
            pbar.set_description(f"Processing chapter {chapter}")
        for page in range(start, end+1):
            if page == start: 
                list(string.ascii_lowercase)
            extract_formulas(images[page], chapter)
            pbar.update(1)

Processing chapter 20: 100%|██████████████████████| 9/9 [00:19<00:00,  2.16s/it]


In [ ]:
pattern = r"^\d+\.\d+ (Theorem|Definition|Definitions|Proposition|Corollary|Remark)"
compiled_pattern = re.compile(pattern, re.MULTILINE)
matches = [m.start(0) for m in re.finditer(compiled_pattern, document_content)]
chunks = [document_content[start:end].strip() for start, end in zip(matches[:-1], matches[1:])]
chunks.append(document_content[matches[-1]:])